In [18]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast'

In [19]:
url = (
       'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/'
       'data/simulated_items_sales.csv'
)

In [20]:
import pandas as pd
import numpy as np
import pytest


def fetch_dataset(
    name : str,
    version: str = 'latest',
    raw: bool = False,
    kwargs_read_csv: dict = {},
    verbose: bool = True
) -> pd.DataFrame:
    """
    Fetch a dataset from the skforecast repository.

    Parameters
    ----------
    name: str
        Name of the dataset to fetch.
    version: str, int, default `'latest'`
        Version of the dataset to fetch. If 'latest', the lastest version will be 
        fetched (the one in the master branch). For a list of available versions, 
        see the repository branches.
    raw: bool, default `False`
        If True, the raw dataset is fetched. If False, the preprocessed dataset 
        is fetched. The preprocessing consists of setting the column with the 
        date/time as index and converting the index to datetime. A frequency is 
        also set to the index.
    kwargs_read_csv: dict, default `{}`
        Kwargs to pass to pandas `read_csv` function.
    verbose: bool, default `True`
        If True, print information about the dataset.
    
    Returns
    -------
    df: pandas DataFrame
        Dataset.
    
    """

    version = 'master' if version == 'latest' else f'{version}'

    datasets = {
        'h2o': {
            'url': 'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/data/h2o.csv',
            'sep': ',',
            'index_col': 'fecha',
            'date_format': '%Y-%m-%d',
            'freq': 'MS',
            'description': (
                'Monthly expenditure ($AUD) on corticosteroid drugs that the Australian '
                'health system had between 1991 and 2008.\nObtained from the book: '
                'Forecasting: Principles and Practice by Rob J Hyndman and George Athanasopoulos.'
            )
        },
        'items_sales': {
            'url': (
                'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/'
                'data/simulated_items_sales.csv'
            ),
            'sep': ',',
            'index_col': 'date',
            'date_format': '%Y-%m-%d',
            'freq': 'D',
            'description': 'Simulated time series for the sales of 3 different items.'
        },
        'air_pollution': {
            'url' : (
                'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/'
                'data/guangyuan_air_pollution.csv'
            ),
            'sep': ',',
            'index_col': 'date',
            'date_format': '%Y-%m-%d',
            'freq': 'D',
            'description': ''
        }
    }
    
    if name not in datasets.keys():
        raise ValueError(
            f"Dataset {name} not found. Available datasets are: {list(datasets.keys())}"
        )
    
    url = datasets[name]['url']
    if version != 'latest':
        url = url.replace('master', f'{version}')

    try:
        sep = datasets[name]['sep']
        df = pd.read_csv(url, sep=sep, **kwargs_read_csv)
    except:
        raise ValueError(
            f"Error reading dataset {name} from {url}. Try to version = 'latest'"
        )

    if not raw:
        index_col = datasets[name]['index_col']
        freq = datasets[name]['freq']
        date_format = datasets[name]['date_format']
        df = df.set_index(index_col)
        df.index = pd.to_datetime(df.index, format=date_format)
        df.index.freq = freq
        df = df.sort_index()
    
    if verbose:
        print(name)
        print('-'*len(name))
        print(datasets[name]['description'])
        print(f"Shape of the dataset: {df.shape}")

    return df


def test_fetch_dataset():
    """
    Test function `fetch_dataset`.
    """
    
    # Test fetching the 'h2o' dataset
    df = fetch_dataset('h2o', version='latest', raw=False, verbose=False)
    assert isinstance(df, pd.DataFrame)
    assert df.shape == (204, 1)
    assert df.index.freq == 'MS'
    assert df.index[0].strftime('%Y-%m-%d') == '1991-07-01'
    assert df.index[-1].strftime('%Y-%m-%d') == '2008-06-01'

    # Test fetching the 'items_sales' dataset
    df = fetch_dataset('items_sales', version='latest', raw=False, verbose=False)
    assert isinstance(df, pd.DataFrame)
    assert df.shape == (1097, 3)
    assert df.index.freq == 'D'
    assert df.index[0] == pd.Timestamp('2012-01-01')
    assert df.index[-1] == pd.Timestamp('2015-01-01')

    # Test fetching a non-existent dataset
    with pytest.raises(ValueError):
        fetch_dataset('non_existent_dataset', version='latest', raw=False, verbose=False)

    # Test fetching a dataset with a non-existent version
    with pytest.raises(ValueError):
        fetch_dataset('h2o', version='non_existent_version', raw=False, verbose=False)

In [21]:
test_fetch_dataset()

In [27]:
fetch_dataset('h2o', version='0.00', raw=False, verbose=True).index[0]

ValueError: Error reading dataset h2o from https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/0.00/data/h2o.csv. Try to version = 'latest'

In [22]:
def load_demo_dataset() -> pd.Series:
    """
    Load demo data set with monthly expenditure ($AUD) on corticosteroid drugs that
    the Australian health system had between 1991 and 2008. Obtained from the book:
    Forecasting: Principles and Practice by Rob J Hyndman and George Athanasopoulos.
    Index is set to datetime with monthly frequency and sorted.

    Parameters
    ----------
    None

    Returns
    -------
    df: pandas Series
        Dataset.
    
    """

    url = (
        'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/'
        'data/h2o.csv'
    )

    df = pd.read_csv(url, sep=',', header=0, names=['y', 'datetime'])
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d')
    df = df.set_index('datetime')
    df = df.asfreq('MS')
    df = df['y']
    df = df.sort_index()

    return df

# create unit tests for load_demo_dataset

def test_load_demo_dataset():
    """
    Test load_demo_dataset function.
    """
    df = load_demo_dataset()
    assert isinstance(df, pd.Series)
    assert df.index.freq == 'MS'
    assert df.index.is_monotonic_increasing
    assert df.index[0] == pd.Timestamp('1991-07-01')
    assert df.index[-1] == pd.Timestamp('2008-06-01')
    assert df.shape == (204,)

# Create 

In [23]:
test_load_demo_dataset()

In [24]:
load_demo_dataset()

datetime
1991-07-01    0.429795
1991-08-01    0.400906
1991-09-01    0.432159
1991-10-01    0.492543
1991-11-01    0.502369
                ...   
2008-02-01    0.761822
2008-03-01    0.649435
2008-04-01    0.827887
2008-05-01    0.816255
2008-06-01    0.762137
Freq: MS, Name: y, Length: 204, dtype: float64

In [25]:
fetch_dataset(name='h2o', raw=False, version="latest")

h2o
---
Monthly expenditure ($AUD) on corticosteroid drugs that the Australian health system had between 1991 and 2008.
Obtained from the book: Forecasting: Principles and Practice by Rob J Hyndman and George Athanasopoulos.
Shape of the dataset: (204, 1)


,x
fecha,
1991-07-01,0.429795
1991-08-01,0.400906
1991-09-01,0.432159
1991-10-01,0.492543
1991-11-01,0.502369
...,...
2008-02-01,0.761822
2008-03-01,0.649435
2008-04-01,0.827887


In [7]:
load_demo_dataset()

datetime
1991-07-01    0.429795
1991-08-01    0.400906
1991-09-01    0.432159
1991-10-01    0.492543
1991-11-01    0.502369
                ...   
2008-02-01    0.761822
2008-03-01    0.649435
2008-04-01    0.827887
2008-05-01    0.816255
2008-06-01    0.762137
Freq: MS, Name: y, Length: 204, dtype: float64